## ADVANCED SQL

In [1]:
import os
if os.path.exists("chinook.db"):
   os.remove('chinook.db')

import sqlite3
conn = sqlite3.connect('chinook.db')
cur=conn.cursor()

In [2]:
def execute(queryString):
  conn.execute(queryString)
  conn.commit()
  
  
def executemany(queryString,data):
  cur.executemany(queryString,data)
  conn.commit()

def showCursorData(cursor):
  print("\n")
  print("Result:")  
  for row in cursor:
    print(row)
  

def showData(tbl):
  query = 'SELECT * FROM {} ORDER BY 1'.format(tbl)
  cursor = conn.execute(query)
  for row in cursor:
    print(row)



### Let's cover joining tables with three or more tables and using sub-queries to join. To proceed with, lets create the appropriate base tables and populate data to it. 

In [3]:

conn.execute("""DROP table IF EXISTS locations""")
conn.execute("""DROP table IF EXISTS departments""")
conn.execute("""DROP table IF EXISTS jobs""")
conn.execute("""DROP table IF EXISTS employees""")

cur.execute("""
CREATE TABLE locations (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    street_address text,
    postal_code text,
    city text NOT NULL,
    state_province text
)""")

cur.execute(""" 
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    department_name text NOT NULL,
    location_id INTEGER NOT NULL,
    FOREIGN KEY (location_id) REFERENCES locations (location_id) ON DELETE CASCADE ON UPDATE CASCADE
)""")

cur.execute(""" 
CREATE TABLE jobs (
    job_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    job_title text NOT NULL,
    min_salary double NOT NULL,
    max_salary double NOT NULL
)""")

cur.execute(""" 
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    first_name text,
    last_name text NOT NULL,
    email text NOT NULL,
    phone_number text,
    hire_date text NOT NULL,
    job_id INTEGER NOT NULL,
    salary double NOT NULL,
    manager_id INTEGER,
    department_id INTEGER NOT NULL,
    FOREIGN KEY (job_id) REFERENCES jobs (job_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (department_id) REFERENCES departments (department_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (manager_id) REFERENCES employees (employee_id) ON DELETE CASCADE ON UPDATE CASCADE
)""")

conn.commit()

In [4]:

# insert data to each table
loc_ins="INSERT INTO locations(location_id,street_address,postal_code,city,state_province) VALUES (?,?,?,?,?)"
loc_data=[(1400,'2014 Jabberwocky Rd','26192','Southlake','Texas'),
         (1500,'2011 Interiors Blvd','99236','South San Francisco','California')]
executemany(loc_ins,loc_data)

dep_ins="INSERT INTO departments(department_id,department_name,location_id) VALUES (?,?,?)"
dep_data=[(1,'Administration',1400),(2,'Marketing',1500),(3,'Purchasing',1500)]
executemany(dep_ins,dep_data)


job_ins="INSERT INTO jobs(job_id,job_title,min_salary,max_salary) VALUES (?,?,?,?)"
job_data = [(1,'Public Accountant',4200.00,9000.00),
            (2,'Accounting Manager',8200.00,16000.00),
            (3,'Accountant',4200.00,9000.00),
            (4,'Finance Manager',8200.00,16000.00)]
executemany(job_ins,job_data)

emp_ins="INSERT INTO employees(employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id) VALUES (?,?,?,?,?,?,?,?,?,?)"
emp_data=[(100,'Steven','King','steven.king@xyz.com','515.123.4567','1987-06-17',4,24000.00,100,1),
         (101,'Neena','Kochhar','neena.kochhar@xyz.com','515.123.4568','1989-09-21',3,17000.00,100,2),
         (102,'Lex','De Haan','lex.de haan@xyz.com','515.123.4569','1993-01-13',3,17000.00,100,3),
         (103,'Alexander','Hunold','alexander.hunold@xyz.com','590.423.4567','1990-01-03',1,9000.00,102,2),
         (104,'Bruce','Ernst','bruce.ernst@xyz.com','590.423.4568','1991-05-21',2,6000.00,103,1),
         (105,'David','Austin','david.austin@xyz.com','590.423.4569','1997-06-25',1,4800.00,103,4)]
executemany(emp_ins,emp_data)

In [5]:
showData('employees')

(100, 'Steven', 'King', 'steven.king@xyz.com', '515.123.4567', '1987-06-17', 4, 24000.0, 100, 1)
(101, 'Neena', 'Kochhar', 'neena.kochhar@xyz.com', '515.123.4568', '1989-09-21', 3, 17000.0, 100, 2)
(102, 'Lex', 'De Haan', 'lex.de haan@xyz.com', '515.123.4569', '1993-01-13', 3, 17000.0, 100, 3)
(103, 'Alexander', 'Hunold', 'alexander.hunold@xyz.com', '590.423.4567', '1990-01-03', 1, 9000.0, 102, 2)
(104, 'Bruce', 'Ernst', 'bruce.ernst@xyz.com', '590.423.4568', '1991-05-21', 2, 6000.0, 103, 1)
(105, 'David', 'Austin', 'david.austin@xyz.com', '590.423.4569', '1997-06-25', 1, 4800.0, 103, 4)


In [6]:
showData('jobs')

(1, 'Public Accountant', 4200.0, 9000.0)
(2, 'Accounting Manager', 8200.0, 16000.0)
(3, 'Accountant', 4200.0, 9000.0)
(4, 'Finance Manager', 8200.0, 16000.0)


In [7]:
showData('departments')

(1, 'Administration', 1400)
(2, 'Marketing', 1500)
(3, 'Purchasing', 1500)


In [8]:
showData('locations')

(1400, '2014 Jabberwocky Rd', '26192', 'Southlake', 'Texas')
(1500, '2011 Interiors Blvd', '99236', 'South San Francisco', 'California')


### COMPLEX JOINS


In [9]:
query="""
SELECT
  e.employee_id AS "Employee #"
  , e.first_name || ' ' || e.last_name AS "Name"
  , e.email AS "Email"
  , e.phone_number AS "Phone"
  , e.hire_date AS "Hire Date"
  , '$' || cast(e.salary as int) AS "Salary"
  , 'works as ' || j.job_title || ' in ' || d.department_name || ' department (manager: '
    || m.first_name || ' ' || m.last_name || ')' AS "Current Job"
   , l.street_address || ', ' || l.postal_code || ', ' || l.city || ', ' || l.state_province AS "Location"
   
FROM employees e
-- to get title of current job_id
  JOIN jobs j 
    ON e.job_id = j.job_id
-- to get name of current manager_id
  LEFT JOIN employees m 
    ON e.manager_id = m.employee_id
-- to get name of current department_id
  LEFT JOIN departments d 
    ON d.department_id = e.department_id
-- to get name of location
  LEFT JOIN locations l
    ON d.location_id = l.location_id
ORDER BY e.employee_id

"""
cursor = conn.execute(query)
showCursorData(cursor)

#Note: Employee David Austin has no department associated , hence results to None in the output.



Result:
(100, 'Steven King', 'steven.king@xyz.com', '515.123.4567', '1987-06-17', '$24000', 'works as Finance Manager in Administration department (manager: Steven King)', '2014 Jabberwocky Rd, 26192, Southlake, Texas')
(101, 'Neena Kochhar', 'neena.kochhar@xyz.com', '515.123.4568', '1989-09-21', '$17000', 'works as Accountant in Marketing department (manager: Steven King)', '2011 Interiors Blvd, 99236, South San Francisco, California')
(102, 'Lex De Haan', 'lex.de haan@xyz.com', '515.123.4569', '1993-01-13', '$17000', 'works as Accountant in Purchasing department (manager: Steven King)', '2011 Interiors Blvd, 99236, South San Francisco, California')
(103, 'Alexander Hunold', 'alexander.hunold@xyz.com', '590.423.4567', '1990-01-03', '$9000', 'works as Public Accountant in Marketing department (manager: Lex De Haan)', '2011 Interiors Blvd, 99236, South San Francisco, California')
(104, 'Bruce Ernst', 'bruce.ernst@xyz.com', '590.423.4568', '1991-05-21', '$6000', 'works as Accounting Ma

### CORRELATED SUBQUERIES

**WHAT IS..**
Correlated subqueries are used for row-by-row processing. Each subquery is executed once for every row of the outer query.

**Difference between Correlated and Nested Subqueries** <br/>

Correlated : Fetch by the outer query, execute the inner query using the value of the outer query, use the values resulting from the inner query to qualify or disqualify the outer query . A Corelated Sub-query should refer its parent(main Query) Table in it .


Nested subquery : The inner query executes first and finds a value, the outer query executes once using the value from the inner query 

**Let's see an example:**

In [10]:
# Correlated 
# Query To Find all employees whose salary is above average for their department


query="""select employee_id,salary,department_id
from employees e 
where salary>=(select avg(salary) 
           from employees where department_id=e.department_id)

"""
cursor = conn.execute(query)
showCursorData(cursor)
           



Result:
(100, 24000.0, 1)
(101, 17000.0, 2)
(102, 17000.0, 3)
(105, 4800.0, 4)


In [11]:
# Nested subquery
query="""select first_name, salary 
from employees  where salary > (select salary
                       from employees where first_name like '%Alex%')"""
cursor = conn.execute(query)
showCursorData(cursor)
           



Result:
('Steven', 24000.0)
('Neena', 17000.0)
('Lex', 17000.0)


### VIEWS <BR/>
A view is nothing more than a SQL statement that is stored in the database with an associated name. A view is actually a composition of a table in the form of a predefined SQL query.

*NOTE : Sqlite supports only CREATE & DROP VIEWS*

**CREATE VIEW**<br/>
Let us create a view from our location table . 

In [12]:
conn.execute('''CREATE VIEW LOCATION_VIEW AS
SELECT postal_code,city,state_province
FROM  LOCATIONS ''')

In [13]:
# Now, you can query LOCATION_VIEW in a similar way as you query an actual table. Following is an example for the same.

query="""SELECT * FROM LOCATION_VIEW"""
cursor = conn.execute(query)
showCursorData(cursor)
   



Result:
('26192', 'Southlake', 'Texas')
('99236', 'South San Francisco', 'California')


### WITH CHECK OPTION
The WITH CHECK OPTION is a CREATE VIEW statement option. The purpose of the WITH CHECK OPTION is to ensure that all UPDATE and INSERTs satisfy the condition(s) in the view definition.

**Example :<br/>
CREATE VIEW EMPLOYEES_VIEW AS <br/>
SELECT first_name,salary <br/>
FROM  employees <br/>
WHERE salary IS NOT NULL <br/>
WITH CHECK OPTION;** <br/>

The WITH CHECK OPTION in this case should deny the entry of any NULL values in the view's salary column, because the view is defined by data that does not have a NULL value in the salary column.


### UPDATING VIEW

A view can be updated under certain conditions which are given below −

The SELECT clause may not contain the keyword DISTINCT.

The SELECT clause may not contain summary functions.

The SELECT clause may not contain set functions.

The SELECT clause may not contain set operators.

The SELECT clause may not contain an ORDER BY clause.

The FROM clause may not contain multiple tables.

The WHERE clause may not contain subqueries.

The query may not contain GROUP BY or HAVING.

Calculated columns may not be updated.

All NOT NULL columns from the base table must be included in the view in order for the INSERT query to function.

**Example:<br/>
UPDATE  EMPLOYEES_VIEW <br/>
   SET salary = 35000<br/>
   WHERE first_name like '%David'<br/>**
   

### DROPPING VIEWS
To drop a view, simply use the DROP VIEW statement with the view_name. 


In [0]:


conn.execute("""DROP VIEW EMPLOYEES_VIEW""")

query="""SELECT * FROM EMPLOYEES_VIEW"""
cursor = conn.execute(query)
showCursorData(cursor)
   

OperationalError: ignored